In [1]:
import tensorflow as tf
import tensorflow.examples.tutorials.mnist.input_data as input_data

In [2]:
mnist = input_data.read_data_sets("G:/dldata/MNIST_data/",one_hot=True)

Extracting G:/dldata/MNIST_data/train-images-idx3-ubyte.gz
Extracting G:/dldata/MNIST_data/train-labels-idx1-ubyte.gz
Extracting G:/dldata/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting G:/dldata/MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
#定义全连接层函数
def fcn_layer(inputs,             #输入数据
              input_dim,          #输入神经元数量
              output_dim,         #输出神经元数量
              activation=None):   #激活函数
    W = tf.Variable(tf.truncated_normal([input_dim, output_dim], stddev=0.1)) #以截断正态分布的随机数初始化W
    b = tf.Variable(tf.zeros(output_dim))  #以0初始化b
    
    XWb = tf.matmul(inputs, W) + b  #j建立表达式：inputs * W + b
    
    if activation is None:  #默认不使用激活函数
        outputs = XWb    #若传入激活函数，则对输出结果进行变换
    else:
        outputs = activation(XWb)
    return outputs

In [4]:
#构建输入层
x = tf.placeholder(tf.float32, [None, 784], name="X")

In [5]:
image_shaped_input = tf.reshape(x, [-1,28,28,1])  #-1代表不确定输入数据有多少行，28*28位图，1个色彩单元
tf.summary.image('input', image_shaped_input, 10) #将输入的样本图像加入到summary中,最多显示10幅图

<tf.Tensor 'input:0' shape=() dtype=string>

In [6]:
#构建隐藏层
h1 = fcn_layer(inputs = x,
               input_dim = 784,
               output_dim = 256,
               activation = tf.nn.relu)

In [7]:
#构建输出层
forward = fcn_layer(inputs=h1,
                    input_dim=256,
                    output_dim=10,
                    activation=None)
pred = tf.nn.softmax(forward)

In [8]:
#定义标签数据占位符
y = tf.placeholder(tf.float32, [None, 10], name="Y")

In [9]:
#前向输出值在以直方图显示
tf.summary.histogram('forward', forward) #定义好前向计算操作后加入summary

<tf.Tensor 'forward:0' shape=() dtype=string>

In [10]:
loss_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=forward, labels=y))

In [11]:
#将loss损失以标量显示
tf.summary.scalar('loss', loss_function) #定义好loss_function后加入summary

<tf.Tensor 'loss:0' shape=() dtype=string>

In [12]:
correct_prediction = tf.equal(tf.arg_max(pred, 1), tf.arg_max(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [13]:
#将accuracy准确率以标量显示
tf.summary.scalar('accuracy', accuracy)

<tf.Tensor 'accuracy:0' shape=() dtype=string>

In [14]:
train_epochs = 40
batch_size = 50
total_batch = int(mnist.train.num_examples/batch_size)
display_step = 1
learning_rate = 0.01

In [15]:
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss_function)

In [16]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [17]:
#合并所有summary
merged_summary_op = tf.summary.merge_all() 
writer = tf.summary.FileWriter('G:/dldata/log/', sess.graph)  #创建写入符

In [18]:
from time import time
startTime = time()

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(train_epochs):
    for batch in range(total_batch):
        xs,ys = mnist.train.next_batch(batch_size)
        sess.run(optimizer, feed_dict={x:xs, y:ys})
        
        #生成summary
        summary_str = sess.run(merged_summary_op, feed_dict={x: xs, y: ys})  #生成摘要信息
        writer.add_summary(summary_str, epoch) #将summary写入文件
        
    loss,acc = sess.run([loss_function,accuracy],
                        feed_dict={x:mnist.validation.images,
                                   y:mnist.validation.labels})
    if (epoch+1) % display_step == 0:
        print("Train Epoch:", '%02d'% (epoch+1), "loss=", "{:.9f}".format(loss),"Accuracy=","{:.4}".format(acc))

duration = time() - startTime
print("Train Finished takes:","{:.2f}".format(duration))

Train Epoch: 01 loss= 0.129277289 Accuracy= 0.967
Train Epoch: 02 loss= 0.134355709 Accuracy= 0.965
Train Epoch: 03 loss= 0.139012754 Accuracy= 0.9674
Train Epoch: 04 loss= 0.187854812 Accuracy= 0.9542
Train Epoch: 05 loss= 0.158874273 Accuracy= 0.9708
Train Epoch: 06 loss= 0.166725457 Accuracy= 0.9676
Train Epoch: 07 loss= 0.191348016 Accuracy= 0.966
Train Epoch: 08 loss= 0.233119920 Accuracy= 0.9608
Train Epoch: 09 loss= 0.198247001 Accuracy= 0.969
Train Epoch: 10 loss= 0.302173704 Accuracy= 0.9618
Train Epoch: 11 loss= 0.247529536 Accuracy= 0.9654
Train Epoch: 12 loss= 0.265054792 Accuracy= 0.9686
Train Epoch: 13 loss= 0.274540693 Accuracy= 0.9656
Train Epoch: 14 loss= 0.335629672 Accuracy= 0.9652
Train Epoch: 15 loss= 0.300855786 Accuracy= 0.9684
Train Epoch: 16 loss= 0.249081448 Accuracy= 0.97
Train Epoch: 17 loss= 0.355799228 Accuracy= 0.971
Train Epoch: 18 loss= 0.396937996 Accuracy= 0.9658
Train Epoch: 19 loss= 0.373554289 Accuracy= 0.9676
Train Epoch: 20 loss= 0.323371559 Accu